In [ ]:
import os
from pathlib import Path

import pandas as pd

from utils import *

In [ ]:
base_path = Path(os.environ["SCRATCH"]) / "DESI-II-VI-results" 
vi_path = base_path / "20230122"

In [ ]:
#read in all files
vi = []
for f in vi_path.glob("*.csv"):
    vi_single = pd.read_csv(f, delimiter = ",", engine='python', keep_default_na=False)
    vi.append(vi_single)
vi = pd.concat(vi)

vi_gp = vi.groupby(['TARGETID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')

In [ ]:
# ### Adding useful columns.  Automatically decide on best_z, spectype, and quality where possible.  Concatenate issues and comments.
choose_best_z(vi)
choose_best_spectype(vi)
choose_best_quality(vi)
concatenate_all_issues(vi)
concatenate_all_comments(vi)
add_extra_details(vi)

In [ ]:
vi.columns

In [ ]:
## Make table with average score

merged_quality = vi.groupby("TARGETID")['VI_quality'].mean()

In [ ]:
merged_quality.to_csv(base_path/"merged_vi.csv")

### look at score conflicts

In [ ]:
conflicts = vi.groupby("TARGETID")['VI_quality'].std()

conflicts = conflicts[np.isfinite(conflicts)] #Nan STD because of one VI

conflicts = conflicts[conflicts>0] # all VI agree

In [ ]:
conflicted_cat = vi[np.isin(vi["TARGETID"], conflicts.index)]

In [ ]:
not_clear = conflicted_cat.groupby("TARGETID")['VI_quality'].mean()
not_clear = not_clear[(not_clear<3) & (not_clear>2)]

conflicted_cat = vi[np.isin(vi["TARGETID"], not_clear.index)]

In [ ]:
conflicted_cat = conflicted_cat.groupby("TARGETID")["VI_scanner"].apply(list)
vi_doers = list(conflicted_cat)
not_jan = ["JAN" not in x for x in vi_doers]
not_jan = conflicted_cat[not_jan]
conflicted_cat = vi[np.isin(vi["TARGETID"], not_jan.index)]

In [ ]:
np.unique(conflicted_cat["TARGETID"]).shape

In [ ]:
conflicted_cat.to_csv(base_path / "conflicted_cat.csv")

### Resolve conflicts and make final table with success indicator

In [ ]:
resolved_cat = merged_quality[(merged_quality>=3) | (merged_quality<=2)]

In [ ]:
resolved_conflicts = pd.read_csv(base_path / "desi-vi_VI_conflicts_DEY.csv")
resolved_conflicts.set_index(keys="TARGETID", inplace=True)

resolved_cat = pd.concat([resolved_conflicts["VI_quality"],resolved_cat])

In [ ]:
conflicted_cat = vi[np.isin(vi["TARGETID"], not_clear.index)]

conflicted_cat = conflicted_cat.groupby("TARGETID")["VI_scanner"].apply(list)
vi_doers = list(conflicted_cat)
jan = ["JAN" in x for x in vi_doers]
jan = conflicted_cat[jan]
jeff_cat = vi[np.isin(vi["TARGETID"], jan.index)]
jeff_cat = jeff_cat[jeff_cat["VI_scanner"]=="JAN"]
jeff_cat.set_index(keys="TARGETID", inplace=True)

In [ ]:
resolved_cat = pd.concat([jeff_cat["VI_quality"],resolved_cat])

In [ ]:
resolved_cat.to_csv(base_path / "merged_vi_resolved.csv")